In [22]:
import tensorflow as tf
#from keras import backend as K
#from keras.layers import Input, BatchNormalization, LSTM, Dense, concatenate, Conv2D, MaxPooling2D, Flatten
#from keras.optimizers import Adam, SGD
#from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

import numpy as np
import random
import matplotlib.pyplot as plt
import os
import cv2
import json
import pathlib

In [23]:
from model.model_fn import model_fn
from model.utils import Params

In [24]:
model_dir = 'experiments/batch_hard'
data_dir = './data/fashion/'

# Load the parameters from json file
json_path = os.path.join(model_dir, 'params-kjh.json')
params = Params(json_path)

In [25]:
'''
data_root = pathlib.Path(data_dir)
all_image_paths = list(data_root.glob('**/*.jpg'))                                     #이미지 수준 파일 경로
all_directories = {'/'.join(str(i).split("/")[:-1]) for i in all_image_paths}    #폴더 수준 파일 경로

labels = list(i.split("/")[-1] for i in all_directories)
'''

'\ndata_root = pathlib.Path(data_dir)\nall_image_paths = list(data_root.glob(\'**/*.jpg\'))                                     #이미지 수준 파일 경로\nall_directories = {\'/\'.join(str(i).split("/")[:-1]) for i in all_image_paths}    #폴더 수준 파일 경로\n\nlabels = list(i.split("/")[-1] for i in all_directories)\n'

In [26]:
'''
IMG_WIDTH, IMG_HEIGHT = 150, 150
'''

'\nIMG_WIDTH, IMG_HEIGHT = 150, 150\n'

In [27]:
'''
def get_label_index(s):
        return labels.index(s.numpy().decode("utf-8").split("/")[-2])
    
def preprocess_image(image):
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.math.divide(image, 255.0)     
        return tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])
    
def load_and_preprocess_image(path):
        image = tf.io.read_file(path)
        return tf.py_function(preprocess_image, [image], tf.float32), tf.py_function(get_label_index, [path], tf.int64)
    
def train_input_fn(data_dir, params):
    datasets = [tf.data.Dataset.list_files("{}/*.jpg".format(image_dir), shuffle=False) for image_dir in all_directories]
    # Create the list of datasets creating filenames
    num_labels = len(all_directories)
    num_classes_per_batch = params.num_classes_per_batch
    num_images_per_class = params.num_images_per_class
    
    def generator():
        while True:
            # Sample the labels that will compose the batch
            labels = np.random.choice(range(num_labels),
                                      num_classes_per_batch,
                                      replace=False)
            for label in labels:
                for _ in range(num_images_per_class):
                    yield label
    

    choice_dataset = tf.data.Dataset.from_generator(generator, tf.int64)
    dataset = tf.data.experimental.choose_from_datasets(datasets, choice_dataset)

    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    batch_size = num_classes_per_batch * num_images_per_class

    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(params.num_epochs)
    dataset = dataset.prefetch(1)

    return dataset
'''

'\ndef get_label_index(s):\n        return labels.index(s.numpy().decode("utf-8").split("/")[-2])\n    \ndef preprocess_image(image):\n        image = tf.image.decode_jpeg(image, channels=3)\n        image = tf.image.convert_image_dtype(image, tf.float32)\n        image = tf.math.divide(image, 255.0)     \n        return tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])\n    \ndef load_and_preprocess_image(path):\n        image = tf.io.read_file(path)\n        return tf.py_function(preprocess_image, [image], tf.float32), tf.py_function(get_label_index, [path], tf.int64)\n    \ndef train_input_fn(data_dir, params):\n    datasets = [tf.data.Dataset.list_files("{}/*.jpg".format(image_dir), shuffle=False) for image_dir in all_directories]\n    # Create the list of datasets creating filenames\n    num_labels = len(all_directories)\n    num_classes_per_batch = params.num_classes_per_batch\n    num_images_per_class = params.num_images_per_class\n    \n    def generator():\n        while True:\n

In [28]:
import model.mnist_dataset as mnist_dataset

def mnist_train_input_fn(data_dir, params):
    """Train input function for the MNIST dataset.

    Args:
        data_dir: (string) path to the data directory
        params: (Params) contains hyperparameters of the model (ex: `params.num_epochs`)
    """
    dataset = mnist_dataset.train(data_dir)
    dataset = dataset.shuffle(params.train_size)  # whole dataset into the buffer
    dataset = dataset.repeat(params.num_epochs)  # repeat for multiple epochs
    dataset = dataset.batch( params.num_classes_per_batch * params.num_images_per_class)
    dataset = dataset.prefetch(1)  # make sure you always have one batch ready to serve
    return dataset

In [29]:
dataset = mnist_train_input_fn(data_dir, params)
print(type(dataset))

<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>


In [30]:
tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.INFO)

# Define the model
tf.logging.info("Creating the model...")
config = tf.estimator.RunConfig(tf_random_seed=230,
                                model_dir=model_dir,
                                save_summary_steps=params.save_summary_steps)
estimator = tf.estimator.Estimator(model_fn, params=params, config=config)

INFO:tensorflow:Creating the model...
INFO:tensorflow:Using config: {'_model_dir': 'experiments/batch_hard', '_tf_random_seed': 230, '_save_summary_steps': 50, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb4a0a48c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
# Train the model
tf.logging.info("Starting training for {} epoch(s).".format(params.num_epochs))
estimator.train(lambda: mnist_train_input_fn(data_dir, params))

INFO:tensorflow:Starting training for 10 epoch(s).
INFO:tensorflow:Calling model_fn.


ValueError: The `pool_size` argument must be a tuple of 2 integers. Received: Tensor("model/block_1/Relu:0", shape=(?, 150, 150, 1), dtype=float32)

# dimension test

In [110]:
tf.reset_default_graph()

In [111]:
num_channels = 32

In [112]:
image = tf.read_file('img/Marled_Knit_Racerback_Tank/img_00000002.jpg')

In [113]:
def preprocess_image(image):
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.math.divide(image, 255.0)     
        return tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])

In [114]:
image = preprocess_image(image)

In [115]:
image = tf.reshape(image, (1, 150, 150, 3))

In [116]:
out = image
channels = [num_channels, num_channels * 2, num_channels, num_channels * 2]
for i, c in enumerate(channels):
    with tf.variable_scope('block_{}'.format(i+1)):
        out = tf.layers.conv2d(out, c, 3, padding='same')
        if params.use_batch_norm:
            out = tf.layers.batch_normalization(out, momentum=bn_momentum, training=is_training)
        out = tf.nn.relu(out)
        out = tf.layers.max_pooling2d(out, 2, 2)

In [117]:
out

<tf.Tensor 'block_4/max_pooling2d/MaxPool:0' shape=(1, 9, 9, 64) dtype=float32>

In [118]:
out.shape[1:] == [9, 9, num_channels * 2]

True

In [24]:
# Evaluate the model on the test set
tf.logging.info("Evaluation on test set.")
res = estimator.evaluate(lambda: test_input_fn(data_dir, params))
for key in res:
    print("{}: {}".format(key, res[key]))

INFO:tensorflow:Creating the model...
INFO:tensorflow:Using config: {'_model_dir': 'experiments/batch_hard', '_tf_random_seed': 230, '_save_summary_steps': 50, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13627c160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Starting training for 20 epoch(s).
Inst

KeyboardInterrupt: 